[Soynlp] 학습 기반 토크나이저 <hr>
- 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저
- 비지도 학습으로 단어 토큰화 ==> 데이터에 자주 등장하는 단어들을 단어로 분석
- 내부적으로 단어 점수 표로 동작

In [1]:
# !pip install soynlp

- [기존 형태소 분석기] : 신조어나 형태소 분석기에 등록되지 않은 단어 제대로 구분하지 함

In [4]:
from konlpy.tag import Okt

tokenizer = Okt()

print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정 입니다.'))

# 형태소 분석시 매개변수 stem = True 설정 : 어간추출 -> 원형을 찾아줌
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정 입니다.',stem = True)) # stem = True -> 어간추출 : 원형을 찾아줌
# 형태소 분석시 매개변수 norm = True 설정 : 정규화 유무
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정 입니다.', norm = True))

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다', '.']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '이다', '.']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다', '.']


- [Sonlpy] 사용 => 말뭉치 데이터셋으로 학습 후 형태소 분석

In [6]:
import os
filename = '../data/text_data.txt'

In [7]:
# => 학습 데이터 처리
from soynlp import DoublespaceLineCorpus # 한개로 통합된 문서 데이터 분리
from soynlp.word import WordExtractor # 단어 추출

In [9]:
# => 훈련 데이터 문서 분리
corpus = DoublespaceLineCorpus(filename)
print(f' 훈련 데이터 문서 : {len(corpus)}')

 훈련 데이터 문서 : 30091


In [10]:
# => Sonlpy 학습 진행
word_extractor = WordExtractor()
# 학습 진행하며 단어별 점수
word_extractor.train(corpus)

training was done. used memory 1.023 Gb


In [12]:
# 단어별 점수표 추출
word_score_table = word_extractor.extract()

all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [18]:
# 단어 별 점수표 확인
for idx, key in enumerate(word_score_table.keys()):
    print(f'[{idx}] - {key}')
    if idx ==30:break

[0] - 샐
[1] - 딥
[2] - 논
[3] - 쇠
[4] - 멜
[5] - 믿
[6] - 섣
[7] - 용
[8] - 랄
[9] - 염
[10] - 잤
[11] - 웹
[12] - 룹
[13] - 높
[14] - 옵
[15] - 벨
[16] - 껏
[17] - 슘
[18] - 찾
[19] - 줬
[20] - 구
[21] - 팸
[22] - 러
[23] - 렛
[24] - 트
[25] - 죽
[26] - 낌
[27] - 율
[28] - 띕
[29] - 캘
[30] - 봅


In [ ]:
## 응집확률 (cohesion probability): 내부 문자열 (substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도
# 원리 : 문자열을 문자 단위로 분리, 왼쪽부터 순서대로 문자를 추가
# 각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계사 / 누적곱 한 값
# 값이 높을수록 : 전체 코퍼스(내가 학습시킨 데이터)에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성이 높음

In [19]:
# 확률
word_score_table['바다'].cohesion_forward

0.06393648140409527

In [20]:
word_score_table['바다에'].cohesion_forward

0.11518621707955429

In [21]:
# soynlpd의 L tokenizer
# 띄어쓰기 단위로 나눈 어절 토큰 : L 토큰 + R 토큰
# 예 ) 공원에 -> 공원 + 에 / 공부하는 -> 공부 + 하는
# 분리 기준 : 점수가 가장 높은 L 토큰을 찾아내는 원리

In [23]:
from soynlp.tokenizer import LTokenizer

# 토큰으로 쪼개기 위한 L토큰 기준값
# 단어를 키로 빼고, 응집력 점수만 밸류 값으로 설정
scores = {word:score.cohesion_forward for word, score in word_score_table.items()}

l_tokenizer = LTokenizer(scores= scores)
l_tokenizer.tokenize('국제사회와 우리의 노력들로 범죄를 척결하자', flatten= False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [25]:
# 최대 점수 토크나이저
# 띄어쓰기가 되지 않는 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저
# 띄어쓰기가 되어 있지 않은 문장을 넣어서 점수를 통해 토큰화 된 결과
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores= scores)
# 공백도 없고, 구두점도 없으면 토큰으로 쪼개지 못함.
# 한 글자씩 쪼개서 확률값을 보고 단어를 예상 -> 반환해줌
# 통계를 기반으로 단어를 추출
# ex ) n 다음에 올 글자는 많고 불확실성이 높다. 그렇지만 net 다음에 올 글자는 제한되고 앞보다 불확실성이 낮다.
# forword - 뒤에 붙을 글자 추측 / backwort - 앞에 붙을 글자 추측
# 명사에다 조사를 추측할땐, 조사는 종류가 많으니까 불확실성이 올라가지만, 조사에다 명사를 추측할 때는 거곳보다 불확실성이 낮아짐.
maxscore_tokenizer.tokenize('국제사회와우리의노력들로범죄를척결하자')

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

In [29]:
# 가장 높은 확률을 가지는 합친 단어 선택
word_score_table['국'].cohesion_forward, word_score_table['국제'].cohesion_forward, word_score_table['국제사'].cohesion_forward, word_score_table['국제사회'].cohesion_forward, word_score_table['국제사회와'].cohesion_forward

(0,
 0.07856876882976202,
 0.09217735975351507,
 0.20075093164820865,
 0.17387399904982392)

In [30]:
# soynlp를 이용한 반복되는 문자 정제
# ㅋㅋ, ㅎㅎ 등의 이모티콘의 경우 불필요하게 연속되는 경우 많음
# ㅋㅋ, ㅋㅋㅋ, ㅋㅋㅋㅋ와 같은 경우를 모두 서로 다른 단어로 처리하는 것은 불필요
# 반복되는 것은 하나로 정규화

In [2]:
from soynlp.normalizer import *
import string

# 구두점 미리 제거
print(string.punctuation)

print(emoticon_normalize('앜^^^^^^^^^',num_repeats=1)) # 반복되는 ㅋ, ㅠ 하나로 처리
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ',num_repeats=1)) # 반복되는 ㅋ, ㅠ 하나로 처리
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ',num_repeats=1)) # 반복되는 ㅋ, ㅠ 두개로 처리
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ',num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ',num_repeats=2))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
앜^^^^^^^^^
아ㅋ영화존잼쓰ㅠ
아ㅋ영화존잼쓰ㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ


In [43]:
# !pip install customized_konlpy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/881.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/881.5 kB ? eta -:--:--
   - ------------------------------------- 41.0/881.5 kB 393.8 kB/s eta 0:00:03
   ---- --------------------------------- 112.6/881.5 kB 930.9 kB/s eta 0:00:01
   --------- ------------------------------ 204.8/881.5 kB 1.1 MB/s eta 0:00:01
   ---------------- ----------------------- 368.6/881.5 kB 1.6 MB/s eta 0:00:01
   ---------------------------- ----------- 624.6/881.5 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 881.5/881.5 kB 2.8 MB/s eta 0:00:00


In [44]:
from ckonlpy.tag import Twitter

# 말뭉치를 가지고 존재하는 단어를 가지고 형태소 분석해줌 -> 신조어나 이름, 조사는 분리 제대로 못함
twitter = Twitter()

twitter.morphs('은경이는 사무실로 갔습니다.')  # 은경이는 이름인데 하나씩 다 분리됨 -> 사전에 단어 추가 -> 해결

c:\ProgramData\anaconda3\envs\TORCH_NLP38\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['은', '경이', '는', '사무실', '로', '갔습니다', '.']

In [45]:
# 형태소 분석기에 사전 추가
twitter.add_dictionary('은경이',tag = 'Noun')

In [46]:
twitter.morphs('은경이는 사무실로 갔습니다.') # 은경이가 더이상 나누어지지 않음!!

['은경이', '는', '사무실', '로', '갔습니다', '.']